<a href="https://colab.research.google.com/github/KarthiK-ctrl-A/BigDataAnalytics/blob/main/S%26P_500_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data = pd.read_csv('/content/1_min_SPY_2008-2021.csv')

data['date'] = pd.to_datetime(data['date'])  # Convert date to datetime format
# Remove unnecessary columns
data = data.drop(columns=['Unnamed: 0', 'barCount', 'average'])

# Add a trading day column
data['trading_day'] = data['date'].dt.dayofweek # 0 - Monday ... 7 - Sunday

data.head()

Using device: cuda


,date,open,high,low,close,volume,trading_day
0,2009-05-22 07:30:00,89.45,89.46,89.37,89.37,7872,4
1,2009-05-22 07:31:00,89.38,89.53,89.37,89.50,5336,4
2,2009-05-22 07:32:00,89.51,89.54,89.48,89.49,3349,4
3,2009-05-22 07:33:00,89.49,89.49,89.31,89.34,3495,4
4,2009-05-22 07:34:00,89.33,89.46,89.33,89.39,9731,4


In [4]:
# Step 2: Data Preprocessing for LSTM
def preprocess_for_lstm(data, feature_cols, target_col):
    # Scaling data for LSTM
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data[feature_cols])

    # Split data into features (X) and target (y)
    X = []
    y = []
    for i in range(60, len(data_scaled)):
        X.append(data_scaled[i-60:i, :])  # last 60 timesteps as input
        y.append(data_scaled[i, target_col])  # predict next timestep

    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test, scaler

# Feature columns for LSTM model
feature_cols = ['open', 'high', 'low', 'close', 'volume']
target_col = feature_cols.index('close')

# Preprocess the data for LSTM
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm, scaler = preprocess_for_lstm(data, feature_cols, target_col)

# Step 3: Build LSTM Model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))  # Predict one value (close price)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build and train the LSTM model
lstm_model = build_lstm_model((X_train_lstm.shape[1], X_train_lstm.shape[2]))
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 457s 9ms/step - loss: 0.0011
Epoch 2/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 496s 9ms/step - loss: 5.2410e-04
Epoch 3/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 450s 9ms/step - loss: 5.1055e-04
Epoch 4/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 451s 9ms/step - loss: 4.9099e-04
Epoch 5/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 510s 9ms/step - loss: 4.9714e-04
Epoch 6/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 502s 9ms/step - loss: 4.9539e-04
Epoch 7/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 504s 9ms/step - loss: 4.9724e-04
Epoch 8/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 501s 9ms/step - loss: 4.8049e-04
Epoch 9/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 495s 9ms/step - loss: 4.8921e-04
Epoch 10/10
51770/51770 ━━━━━━━━━━━━━━━━━━━━ 501s 9ms/step - loss: 4.8301e-04


In [5]:
# Step 4: GNN Preprocessing and Model
def preprocess_for_gnn(data):
    # Create graph data for GNN model
    nodes = len(data)
    edges = [(i, i+1) for i in range(nodes - 1)]  # Simple sequential graph

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    x = torch.tensor(data[['open', 'high', 'low', 'close', 'volume']].values, dtype=torch.float)
    y = torch.tensor(data['close'].values, dtype=torch.float)
    return Data(x=x, edge_index=edge_index, y=y)

# Preprocess the data for GNN
gnn_data = preprocess_for_gnn(data)

# Step 5: Build GNN Model
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(5, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc = nn.Linear(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.fc(x)
        return x.squeeze()

# Create GNN model
gnn_model = GCN()
optimizer = optim.Adam(gnn_model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train the GNN model
def train_gnn_model(gnn_model, data, epochs=20):
    gnn_model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = gnn_model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
    return gnn_model

gnn_model = train_gnn_model(gnn_model, gnn_data)


In [6]:
# Step 6: Evaluate LSTM and GNN Models
# def evaluate_model(model, X_test, y_test, scaler=None):
#     predictions = model.predict(X_test)
#     if scaler:
#         predictions = scaler.inverse_transform(predictions)
#         y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
#     mse = mean_squared_error(y_test, predictions)
#     r2 = r2_score(y_test, predictions)
#     return mse, r2

# Step 6: Evaluate LSTM and GNN Models
def evaluate_model(model, X_test, y_test, scaler=None):
    predictions = model.predict(X_test)
    if scaler:
        # Reshape predictions to have the same number of features as the original data
        predictions = predictions.reshape(-1, 1)  # Reshape to (num_samples, 1)
        predictions = scaler.inverse_transform(np.repeat(predictions, 5, axis=1))[:, 0] # Inverse transform and extract the first column (close price)
        # Reshape predictions back to 2D array for compatibility with y_test shape
        predictions = predictions.reshape(-1, 1)
        y_test = scaler.inverse_transform(np.repeat(y_test.reshape(-1, 1), 5, axis=1))[:, 0]  # Inverse transform and extract the first column (close price)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mse, r2

# Evaluate LSTM
lstm_mse, lstm_r2 = evaluate_model(lstm_model, X_test_lstm, y_test_lstm, scaler=scaler)

# Evaluate GNN
gnn_model.eval()
with torch.no_grad():
    gnn_predictions = gnn_model(gnn_data).numpy()
    gnn_mse = mean_squared_error(gnn_data.y.numpy(), gnn_predictions)
    gnn_r2 = r2_score(gnn_data.y.numpy(), gnn_predictions)

# Step 7: Combined Prediction (Simple Averaging)
# Reshape LSTM predictions to have 5 features for inverse_transform
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions_reshaped = np.repeat(lstm_predictions, 5, axis=1)

# Inverse transform and extract the 'close' price (first column)
lstm_predictions_inverse = scaler.inverse_transform(lstm_predictions_reshaped)[:, 0]

# Reshape back to 2D for consistency with GNN predictions
lstm_predictions_inverse = lstm_predictions_inverse.reshape(-1, 1)

# Ensure gnn_predictions and y_test_lstm_inverse have the same shape
y_test_lstm_inverse = scaler.inverse_transform(np.repeat(y_test_lstm.reshape(-1, 1), 5, axis=1))[:, 0]

combined_predictions = (gnn_predictions[:len(lstm_predictions_inverse)] + lstm_predictions_inverse.flatten()) / 2  # Adjust gnn_predictions to match LSTM prediction length
combined_mse = mean_squared_error(y_test_lstm_inverse, combined_predictions)
combined_r2 = r2_score(y_test_lstm_inverse, combined_predictions)

# Step 8: Print Results
print(f"LSTM Model MSE: {lstm_mse}, R2: {lstm_r2}")
print(f"GNN Model MSE: {gnn_mse}, R2: {gnn_r2}")
print(f"Combined Model MSE: {combined_mse}, R2: {combined_r2}")

12943/12943 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step
12943/12943 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step
LSTM Model MSE: 23.187380794629764, R2: 0.9965742502147483
GNN Model MSE: 8785.1650390625, R2: -0.24246740341186523
Combined Model MSE: 7785.408810927518, R2: -0.1502317919542182
